## TF-IDF를 이용한 핵심어 추출
- 형태소 추출하기
    - 한글 텍스트로 전처리
    - 형태소 단위로 추출
- 분류 모델의 학습 데이터로 변환
    - corpus index 생성
    - TF-IDF로 변환

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [28]:
import re
from konlpy.tag import Okt

# 한글만 추출
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('', text)
    return result

# 형태소 단위로 추출
def get_pos(x):
    tagger = Okt()
    pos = tagger.pos(x)
    pos = ['{}/{}'.format(word, tag) for word, tag in pos]
    return pos


In [44]:
df2 = pd.read_csv("review_data.csv")
df2.shape

(715, 3)

In [45]:

df = pd.read_csv("review_data.csv")

# 한글만 추출
df['ko_text'] = df['review'].apply(lambda x: text_cleaning(str(x)))
del df['review']
df = df[df['ko_text'].str.len() > 0]

# 형태소 단위로 추출
result = get_pos(df['ko_text'].values[0])
print(result)

['맛있었어요/Adjective', '콩나물/Noun', '엄청/Adverb', '좋아해서/Adjective', '많이/Adverb', '먹었는데도/Verb', '많이/Adverb', '남았어요/Verb', '조금/Noun', '아까웠/Adjective', '슴/Noun', 'ㅠㅠ/KoreanParticle', '남은건/Verb', '포장/Noun', '도/Josa', '되서/Verb', '너무/Adverb', '좋았습니다/Adjective', '다음/Noun', '엔/Josa', '불고기/Noun', '먹으러/Verb', '가고싶어요/Verb', '너무/Adverb', '맛있게/Adjective', '잘/Verb', '먹었/Verb', '습/Noun', '니당/Verb']


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# 형태소를 벡터 형태의 학습 데이터셋으로 변환
index_vectorizer = CountVectorizer(tokenizer = lambda x: get_pos(x)) # 형태소 분리
X = index_vectorizer.fit_transform(df['ko_text'].to_list()) # 형태소 분리한 말뭉치들을 벡터를 카운트하는 연산 수행
print(X.shape)
print(str(index_vectorizer.vocabulary_)[:100]+"..")
# 형태소 추출 테스트
print(df['ko_text'].values[0])
print(X[0])

# TF-IDF로 변환
# 형태소를 벡터 형태의 학습데이터셋으로 변환
tfidf_vectorizer = TfidfTransformer()
X = tfidf_vectorizer.fit_transform(X)
print(X.shape)
print(X[0])

## 긍정 부정 리뷰 분류하기
- 분류 모델링
    - 데이터셋 분리
    - 로지스틱 회귀

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score

# 훈련 데이터 셋, 테스트 데이터셋 분리
y = df['y']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30) # 7대3으로 분리
print(x_train.shape)
print(x_test.shape)

# 학습
lr = LogisticRegression(random_state=42)
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)
y_pred_probability = lr.predict_proba(x_test)[:, 1]

# 성능 평가
print(accuracy_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))

confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(confmat)


false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred_probability)
roc_auc = roc_auc_score(y_test, y_pred_probability)

plt.rcParams['figure.figsize'] = [5, 4]
plt.plot(false_positive_rate, true_positive_rate, label='ROC curve (area = %0.3f)' % roc_auc, 
         color='red', linewidth=4.0)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate') # 부정 리뷰
plt.ylabel('True Positive Rate') # 긍정 리뷰
plt.title('ROC curve of Logistic regression')
plt.legend(loc="lower right")



## 중요 키워드 분석
- 회귀 모델의 피쳐 영향력 추출
- 중요 피쳐의 형태소
